In [78]:
import stable_baselines3
import gym
import numpy as np

# Environement
import gym
from gym import spaces

# Evaluate the environement
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results

# Agent
from stable_baselines3 import A2C
# Policy
from stable_baselines3.ppo import MlpPolicy

import matplotlib.pyplot as plt
import os

In [85]:
SOLUTION_NAME = "StbBL_Base"
log_dir = SOLUTION_NAME + "/"
os.makedirs(log_dir, exist_ok=False)

FileExistsError: [Errno 17] File exists: 'StbBL_Base/'

# Environement

## Definition

In [62]:
class CustomEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    CASE_VIDE = 255
    
    def __init__(self, grid_size=(10,10),nspecies=8):
        # CONTROLES DONNEES
        assert nspecies < 256-1 # pour compatibilité avec uint8. 0 est pris par l'information case vide
        
        # INITILISATIION SUBCLASS
        super().__init__()
        
        
        # PARAMETRES 
        self.grid_size = grid_size
        self.nspecies = nspecies

        
        
        # Define action space
        # ESSAI 1 : NE MARCHE PAS, CE SONT DES VALEURS CONTINUES MEME AVEC dype
#         self.action_space = spaces.Box(low=np.array([0,0,0]),
#                                       high=np.array(self.grid_size + (self.nspecies,)),
#                                       shape=(3,),
#                                       dtype=np.uint8)        
        # ESSAI 2 :
        self.action_space =spaces.MultiDiscrete(self.grid_size + (self.nspecies,))
        
        # Define observation space
        self.observation_space = spaces.Box(low=0,
                                            high=self.nspecies,
                                            shape=grid_size, dtype=np.uint8)
        
        self._state = np.full(self.grid_size,self.CASE_VIDE)
    
    
    def step(self, action):
        # ACTION -> ACTION LOCALE
        x,y,specie = action
        
        print(x,y,specie)
        
        # CONTROLE
        assert specie != self.CASE_VIDE
        
        done = False # JUSQU'A PREUVE DU CONTRAIRE LA PARTIE CONTINUE
        info = {} # UTILISE POUR DEBUGAGE
        reward = 0
        if self._state[x,y]==self.CASE_VIDE:
            # ACTION VALIDE (CASE LIBRE) CAR ==0            
            self._state[x,y] = specie # ON PLANTE
            reward = 1 # <--------------------------------------------------REWARD NON LEGUME
            if (self._state[x,y]==self.CASE_VIDE).sum()==0:
                # DANS LE CAS ON IL N'Y A PLUS DE CASES VIDES
                reward = 100 # <--------------------------------------------------REWARD SUCCESS
                done = True 
            
        else:
            # ACTION NON VALIDE CASE DEJA REMPLIE
            reward = -100 # <--------------------------------------------------REWARD FAIL
            done = True
            
        return self.to_observation(), reward, done, info
    
    def reset(self):
        self._state = np.full(self.grid_size,self.CASE_VIDE) # TODO ESSAYER DE FAIRE UN RANDOM A CHAQUE RESET PLUTOT 0, COMME SUGGERE PAR GUILLAUME
        return self.to_observation()
    
    def to_observation(self):
        return self._state.copy()
    
    def render(self,mode="human"):
        return self.to_observation()
        


## Instanciation

In [30]:
env = CustomEnv()
eval_env = CustomEnv() # Use a separate environement for evaluation

In [ ]:
class EnvWithQuotas(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    def __init__(self, grid_size=(10,10),nspecies=255):
        # CONTROLES DONNEES
        assert nspecies < 256 # pour compatibilité avec uint8
        assert quotas is None or quotas.shape == (nspecies,)
        
        # INITILISATIION SUBCLASS
        super().__init__()
        
        
        # PARAMETRES 
        self.grid_size = grid_size
        self.nspecies = nspecies
        if self.quotas is None:
            self.quotas = np.zeros((nspecies,))

        
        
        # Define action space
        #    TODO VERIFIER LE TYPE
        self.action_space = spaces.Box(low=np.array([0,0,1]),
                                      high=np.array(self.grid_size + (self.nspecies,)),
                                      shape=(3,),
                                      dtype=np.uint8)        
        # Define observation space
        self.observation_space = spaces.Box(low=0,
                                            high=self.nspecies,
                                            shape=grid_size, dtype=np.uint8)
        
        self.state = np.zeros(self.grid_size)
    
    
    def step(self, action):
        # ACTION -> ACTION LOCALE
        x,y,specie = action
        # CONTROLE
        
        done = False
        if self._state[x,y]==0:
            # ACTION VALIDE CASE LIBRE CAR ==0
            
            self._state[x,y]
            
        else:
            # ACTION NON VALIDE CASE DEJA REMPLIE
            reward = -100 # <--------------------------------------------------REWARD FAIL
            done = True
            

        
        return self.to_observation(), reward, done, info
    
    def reset(self):
        self._state = np.zeros(self.grid_size) # TODO ESSAYER DE FAIRE UN RANDOM A CHAQUE RESET PLUTOT 0, COMME SUGGERE PAR GUILLAUME
        return self.to_observation()
    
    def to_observation(self):
        return self._state.copy()

In [30]:
(np.array([5,1])!=0).sum()

2

In [23]:
v,c = np.unique(np.array([5,1]),return_counts=True)
remp[v] += 1 
remp

array([0., 3., 0., 0., 0., 3., 0., 0., 0., 0.])

# Agent

In [72]:
agent = A2C(MlpPolicy, env, verbose=2)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Evaluate random Agent

In [32]:
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(agent, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:100.00 +/- 0.00


/home/maxime/.pyenv/versions/iarchitect/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


## Train the agent

In [90]:
class MyCallBack(BaseCallback):
    def __init__(self,log_dir):
        super().__init__()
        self.log_dir = log_dir
        
    def _on_step(self) -> bool:
        x, y = ts2xy(load_results(self.log_dir), 'timesteps')
        print(x,y)

In [91]:
# Train the agent for 10000 steps
agent.learn(total_timesteps=1000,callback=MyCallBack(log_dir))

LoadMonitorResultsError: No monitor files of the form *monitor.csv found in StbBL_Base/

In [ ]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [77]:


plot_results([log_dir], timesteps, results_plotter.X_TIMESTEPS, "TD3 LunarLander")
plt.show()

NameError: name 'os' is not defined

In [ ]:
plot_results([log_dir], timesteps, results_plotter.X_TIMESTEPS, "TD3 LunarLander")
plt.show()